In [49]:
import mlflow
import mlflow.keras
from mlflow.types import Schema, TensorSpec
from mlflow.models import ModelSignature

import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [11]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000/")

mlflow.set_experiment('MNIST-experiment')

mlflow.tensorflow.autolog()

2025/04/16 16:32:09 INFO mlflow.tracking.fluent: Experiment with name 'MNIST-experiment' does not exist. Creating a new experiment.


In [13]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [15]:
X_train = X_train.reshape(-1,28,28,1).astype('float') / 255.0
X_test = X_test.reshape(-1,28,28,1).astype('float') / 255.0


In [17]:
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)


In [19]:
model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, kernel_size=(3, 3), activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation="relu"),
        Dropout(0.5),
        Dense(10, activation="softmax")
    ])

In [27]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [51]:
# Parametry treningu
batch_size = 128
epochs = 5

# Rozpoczęcie eksperymentu MLflow
with mlflow.start_run(run_name='CNN_MNIST'):
    # Inicjalizacja modelu

    # Trenowanie modelu na danych treningowych
    history = model.fit(
        X_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_test, y_test),
        verbose=1
    )

    # Ewaluacja modelu
    score = model.evaluate(X_test, y_test, verbose=1)
    print(f"Test loss: {score[0]:.4f}")
    print(f"Test accuracy: {score[1]:.4f}")

    # Logowanie własnych parametrów (opcjonalnie)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("epochs", epochs)
    mlflow.log_metric("final_loss", score[0])
    mlflow.log_metric("final_accuracy", score[1])

    input_schema = Schema([TensorSpec(np.dtype(np.float32), (-1, 28, 28, 3), "input")])
    signature = ModelSignature(inputs=input_schema)
    
    model = mlflow.tensorflow.log_model(
        model, "model",
        keras_model_kwargs={"save_format": "h5"},
        signature=signature
    )


print("Trening zakończony, eksperyment został zarejestrowany w MLflow.")

Epoch 1/5
469/469 [==============================] - 2s 3ms/step - loss: 0.0082 - accuracy: 0.9969 - val_loss: 0.0237 - val_accuracy: 0.9942
Epoch 2/5
469/469 [==============================] - 1s 3ms/step - loss: 0.0071 - accuracy: 0.9975 - val_loss: 0.0328 - val_accuracy: 0.9928
Epoch 3/5
469/469 [==============================] - 1s 3ms/step - loss: 0.0068 - accuracy: 0.9977 - val_loss: 0.0265 - val_accuracy: 0.9942
Epoch 4/5
469/469 [==============================] - 1s 3ms/step - loss: 0.0075 - accuracy: 0.9975 - val_loss: 0.0294 - val_accuracy: 0.9937
Epoch 5/5
1/1 [==============================] - 0s 12ms/step


INFO:tensorflow:Assets written to: C:\Users\MOlsz\AppData\Local\Temp\tmpguxlifnu\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\MOlsz\AppData\Local\Temp\tmpguxlifnu\model\data\model\assets


313/313 [==============================] - 1s 2ms/step - loss: 0.0271 - accuracy: 0.9944
Test loss: 0.0271
Test accuracy: 0.9944
🏃 View run CNN_MNIST at: http://127.0.0.1:5000/#/experiments/530024684035339485/runs/7eb44af33bb64786b11e3b26a674ccfd
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/530024684035339485
Trening zakończony, eksperyment został zarejestrowany w MLflow.
